In [1]:
from transformers import RobertaForMaskedLM, RobertaTokenizer

model_name = "roberta-base"  # Or another suitable RoBERTa model
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForMaskedLM.from_pretrained(model_name)

In [35]:
!pip install python-dotenv

In [37]:
from dotenv import load_dotenv
import os

load_dotenv()

from wandb import login

login(key=os.getenv("WAND_API"))

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [48]:
#from torch.utils.data import Dataset
from datasets import load_dataset
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = load_dataset("text", data_files={"train": "data/mfc/train_data.txt"})["train"]
train_dataset = train_dataset.map(tokenize_function, batched=True)

eval_dataset = load_dataset("text", data_files={"test": "data/mfc/test_data.txt"})["test"]
eval_dataset = eval_dataset.map(tokenize_function, batched=True)


loading file https://huggingface.co/roberta-base/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-base/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-base/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/d53fc0fa09b8342651efd4073d75e19617b3e51287c2a535becda5808a8db287.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
loading file https://huggingface.co/roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggin

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?ba/s]

Using custom data configuration default-ff3ba2de3101ab4e
Reusing dataset text (/root/.cache/huggingface/datasets/text/default-ff3ba2de3101ab4e/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [49]:
for i in range(5):
    print(f"Train article {i+1}:", train_dataset[i])

Train article 1: {'text': 'IMM-29061  PRIMARY  Letters: Humanitarian parole for Haiti quake victims  Re: "From Pa. to Haiti to aid family," June 18:  Thank you very much for your story about the Haitian family who are separated due to immigration regulations. Your story illuminates the plight of this family very clearly. Their problems are ones that we in the American Friends Service Committee see over and over again as we work with Haitian immigrant families in New York and Florida, and as we serve earthquake-displaced families in Haiti itself.  United States immigration policy permits the U.S. Citizenship and Immigration Service (USCIS) to grant humanitarian parole, which is permission for foreign nationals to live in the United States for a defined period of time. These paroles are meant to be used in response to an emergency or natural catastrophe.  Unfortunately, USCIS uses this procedure only very sparingly, even in the event of so glaring a humanitarian disaster as the earthquak

In [50]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True)

In [51]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./models/roberta-mfc",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=24,
    save_steps=5_000,
    eval_steps=2_000,
    logging_steps=100,
    save_total_limit=2,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [52]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: text. If text are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 38450
  Num Epochs = 3
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 4809
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
100,1.239400
200,1.233100
300,1.217000
400,1.229400
500,1.208900
600,1.232300
700,1.219000
800,1.210400
900,1.216300
1000,1.197400




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=4809, training_loss=1.1523752024129612, metrics={'train_runtime': 3488.3888, 'train_samples_per_second': 33.067, 'train_steps_per_second': 1.379, 'total_flos': 3.03676711911936e+16, 'train_loss': 1.1523752024129612, 'epoch': 3.0})

In [53]:
trainer.save_model()

Saving model checkpoint to ./models/roberta-mfc
Configuration saved in ./models/roberta-mfc/config.json
Model weights saved in ./models/roberta-mfc/pytorch_model.bin
